In [11]:
# imports
import json
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import sklearn
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from time import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
city = 'Bhopal'

In [3]:
# data read
df1 = pd.read_excel(f'./data-sept-14/{city}/Item-wise-details-{city}-17-23-Dec-2021.xlsx')
df1 = df1.sort_values(by =['Order_Date', 'Location_Code', 'Order_Number'])
df2 = pd.read_excel(f'./data-sept-14/{city}/Item-wise-details-{city}-24-31-Dec-2021.xlsx')
df2 = df2.sort_values(by =['Order_Date', 'Location_Code', 'Order_Number'])

In [4]:
df = pd.concat([df1, df2])
df = df[pd.notna(df['Oven_Time'])]

In [5]:
RestaurantWiseOrders = {}
for index, order in df.iterrows():
    rest = order['Location_Code']
    if rest in RestaurantWiseOrders:
        RestaurantWiseOrders[rest].append(order)
    else:
        RestaurantWiseOrders[rest] = [order]

orderDict1 = df1.to_dict('records')
orderDict2 = df2.to_dict('records')
orders = []
for dic in orderDict1:
	orders.append(dic)
for dic in orderDict2:
    orders.append(dic)

In [6]:
sampleRest = 'DPI63827'
sampleOrder = RestaurantWiseOrders[sampleRest][0]

In [27]:
sampleOrder

Location_Code                                              DPI63827
storedesc                 C. I. SQUARE BUILDING, KOLAR ROAD, BHOPAL
city                                                         BHOPAL
Order_Date                                      2021-12-17 00:00:00
Order_Number                                                      1
Customer_Code                                                 27166
Customer_Name                         SakshamGawande SakshamGawande
Actual_Order_Date                        2021-12-17 11:05:32.803000
Order_Status_Code                                                 4
Order_Type_Code                                                   D
Order_Saved                              2021-12-17 11:05:32.803000
Order_Time                                                        0
Driver_ID                                                  929444.0
Driver_Shift                                                      1
Route_Time                                      

In [12]:
def getMinuteSlot(dt):
    current = datetime(2011, 1, 1)
    t1 = current.time()
    t2 = dt.time()
    for i in range(0, 24*60):
        current = current + timedelta(minutes = 1)
        if(current.time() > t2):
            return i
    return 24*60-1

In [14]:
restaurantWiseCumulativeOrders = {}
for rest in RestaurantWiseOrders.keys():
    restaurantWiseCumulativeOrders[rest] = {}

# order count per slot
for rest in RestaurantWiseOrders.keys():
    for order in RestaurantWiseOrders[rest]:
        date = order['Order_Date']
        minuteSlot = getMinuteSlot(order['Kitchen_Display_Time'])
        if date in restaurantWiseCumulativeOrders[rest]:
            restaurantWiseCumulativeOrders[rest][date][minuteSlot]+=1
        else:
            t = []
            for i in range(0, 24*60):
                t.append(0)
            restaurantWiseCumulativeOrders[rest][date] = t
            restaurantWiseCumulativeOrders[rest][date][minuteSlot] = 1
            
#prefix sum            
for rest in restaurantWiseCumulativeOrders.keys():
    for date in restaurantWiseCumulativeOrders[rest].keys():
        for i in range(1, 24*60):
            restaurantWiseCumulativeOrders[rest][date][i] += restaurantWiseCumulativeOrders[rest][date][i-1]

In [15]:
d = datetime.strptime('2021-12-17 00:00:00', '%Y-%m-%d %H:%M:%S')
restaurantWiseCumulativeOrders[sampleRest][d]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [72]:
def countPastOrders(order, restaurantWiseCumulativeOrders, slotTime):
    minuteSlot = getMinuteSlot(order['Kitchen_Display_Time'])
    total = restaurantWiseCumulativeOrders[order['Location_Code']][order['Order_Date']][minuteSlot]
    prev = restaurantWiseCumulativeOrders[order['Location_Code']][order['Order_Date']][max(minuteSlot-slotTime, 0)]
    return total-prev

In [73]:
countPastOrders(sampleOrder, restaurantWiseCumulativeOrders, 10)

4

In [17]:
item_types = df['Menu_Code'].unique()
rest_types = df['Location_Code'].unique()

In [18]:
def getOneHotEncodingItem(item, item_types, shouldClubItemsOfSameType=True):
    a = []
    if shouldClubItemsOfSameType:
        a = [i[0:3] for i in item_types]
    else:
        a = item_types
    it = []
    for i in a:
        if i not in it:
            it.append(i)
    ret=[]
    for i in range(0, len(it)):
        if (it[i] == item[0:min(len(item), len(it[i]))]):
            ret.append(1)
        else:
            ret.append(0)
    return ret

In [19]:
def getOneHotEncodingRestaurant(dp, restaurants):
    l = []
    for i in range(0, len(restaurants)):
        if(restaurants[i] == dp):
            l.append(1)
        else:
            l.append(0)
    return l

In [29]:
total_slots = 24
def getTimeIndex(dt):
	current = datetime(2011, 1, 1)
	t1 = current.time()
	t2 = dt.time()
	for i in range(0, total_slots):
		current = current + timedelta(minutes = 60)
		if(current.time() > t2):
			return i
	return total_slots-1

In [21]:
def getOneHotEncodingTimeSlot(ts):
    l = []
    for i in range(0, 24):
        l.append(0)
    l[ts] = 1
    return l

In [28]:
def get_preparation_time(order):
	diff = order['Oven_Time'] - order['Kitchen_Display_Time']
	return diff.total_seconds()

In [87]:
isThreshold = False
threshold = 30 # minutes for preparation time (threshold)
splitMethod = 'random'
slotTime = 10 # minutes
X_train, X_test, y_train, y_test = [], [], [], []
X = []
y = []
for index, order in df.iterrows():
    features = []
    features = getOneHotEncodingItem(order['Menu_Code'], item_types, True)
    features = features + getOneHotEncodingRestaurant(order['Location_Code'], rest_types)
    features = features + getOneHotEncodingTimeSlot(getTimeIndex(order['Kitchen_Display_Time']))
    features = features + [countPastOrders(order, restaurantWiseCumulativeOrders, slotTime)]
    if(isThreshold and (get_preparation_time(order) > (threshold-9)*60)): #removing inputs 
        continue
    X.append(features)
    y.append(420+round(get_preparation_time(order)))
#     if((order['ORDER_DATE'] >='2021-12-20' and order['ORDER_DATE'] <= '2021-12-24') or (order['ORDER_DATE'] >='2021-12-27' and order['ORDER_DATE'] <= '2021-12-29')):
#         X_train.append(features)
#         y_train.append(420+round(get_preparation_time(order)))
#     elif(order['ORDER_DATE'] =='2021-12-30'):
#         X_test.append(features)
#         y_test.append(420+round(get_preparation_time(order)))

In [88]:
if splitMethod == 'random':
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [89]:
#activation: relu, logistic
def ModelResults(X_train, X_test, y_train, y_test, model, isNormalizationRequired=True):
    
    if(isNormalizationRequired):
        scaler = StandardScaler()
        scaler.fit(X_train)

        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
    if(model=='NN'):
        hidden_layer_sizes=(80, 30, 10)
        activation='relu'
        print("Neural Network model")
        print("Hidden layers: ", hidden_layer_sizes)
        print("Activation function: ", activation)
        mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, max_iter=1000, verbose=True, learning_rate='constant', learning_rate_init=0.01)
        mlp.fit(X_train, y_train)
        print(mlp.best_loss_)
        
        predictions = mlp.predict(X_train)
        print("RMSE score on validation set: ", getRMSE(y_train, predictions))
        
        predictions = mlp.predict(X_test)
        print("RMSE score on test set: ", getRMSE(y_test, predictions))
        
    elif(model=='RF'):
        print("Random Forest Model")
        regressor = RandomForestRegressor()
        regressor.fit(X_train, y_train)
        
        predictions = regressor.predict(X_train)
        print("RMSE score on validation set: ", getRMSE(y_train, predictions))
        
        predictions = regressor.predict(X_test)
        print("RMSE score on test set: ", getRMSE(y_test, predictions))
    else:
        print("Model not implemented yet")

In [ ]:
ModelResults(X_train, X_train, y_train, y_test, 'NN')

Neural Network model
Hidden layers:  (80, 30, 10)
Activation function:  relu
Iteration 1, loss = 5.56624583
Iteration 2, loss = 5.41652595
Iteration 3, loss = 5.38271338
Iteration 4, loss = 5.36125980
Iteration 5, loss = 5.34715645
Iteration 6, loss = 5.33566927
Iteration 7, loss = 5.32773323
Iteration 8, loss = 5.32054863
Iteration 9, loss = 5.31434716
Iteration 10, loss = 5.30752506
Iteration 11, loss = 5.30242575
Iteration 12, loss = 5.29682883
Iteration 13, loss = 5.29229817
Iteration 14, loss = 5.29074962
Iteration 15, loss = 5.28493832
Iteration 16, loss = 5.28322644
Iteration 17, loss = 5.28091047
Iteration 18, loss = 5.27822052
Iteration 19, loss = 5.27687497
Iteration 20, loss = 5.27355301
Iteration 21, loss = 5.27205333
Iteration 22, loss = 5.27051831
Iteration 23, loss = 5.26817511
Iteration 24, loss = 5.26731033
Iteration 25, loss = 5.26727854
Iteration 26, loss = 5.26396543
Iteration 27, loss = 5.26336564
Iteration 28, loss = 5.26121637
Iteration 29, loss = 5.25966296
Iter

In [86]:
mlp.predict_log_proba(X)

array([[ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635],
       [ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635],
       [ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635],
       ...,
       [ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635],
       [ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635],
       [ -9.68770891,  -7.18229529,  -7.05802898, ..., -10.05756664,
        -11.47823238,  -9.53518635]])

In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor 

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)
predictions = regressor.predict(X_train)

In [ ]:
mse = sklearn.metrics.mean_squared_error(y_train, predictions)  
rmse = math.sqrt(mse)  
rmse

In [14]:
def getRMSE(actual, predictions):
    mse = sklearn.metrics.mean_squared_error(actual, predictions)
    return math.sqrt(mse)